In [ ]:
import respiration.dataset as repository

dataset = repository.from_default()

# The first 3 subjects are used for training
training_subjects = dataset.get_subjects()[:-1]
training_scenarios = [
    '101_natural_lighting',
    '103_abrupt_changing_lighting',
]

# The last 3 subjects are used for testing
test_subjects = dataset.get_subjects()[-1:]
test_scenarios = [
    '101_natural_lighting',
    '103_abrupt_changing_lighting',
]

In [ ]:
import math
import keras
import itertools
import numpy as np
from scipy.signal import resample
from respiration.utils.unisens import VitalSigns
from respiration.extractor.mtts_can import preprocess_video_frames
import respiration.preprocessing as preprocessing


class DataGenerator(keras.utils.Sequence):
    def __init__(
            self,
            subjects: list[str],
            scenarios: list[str],
            frames_per_video: int,
            dim: tuple[int, int] = (36, 36),
            batch_size: int = 32,
            frame_depth: int = 10,
            shuffle: bool = True):
        super(DataGenerator, self).__init__()

        self.subjects = subjects
        self.scenarios = scenarios
        self.training_scenarios = list(itertools.product(subjects, scenarios))

        self.dim = dim
        self.batch_size = batch_size
        self.frames_per_video = frames_per_video
        self.frame_depth = frame_depth
        self.shuffle = shuffle

    def __len__(self):
        """Number of batches per epoch"""
        return math.ceil(len(self.training_scenarios) / self.batch_size)

    def on_epoch_end(self):
        """Method called at the end of every epoch."""
        # TODO: Implement shuffle
        pass

    def get_ground_truths(self, subject: str, scenario: str) -> tuple[np.ndarray, np.ndarray]:
        """
        Get the respiration rate and pulse rate signals for a given subject and scenario.
        :param subject: 
        :param scenario: 
        :return: 
        """

        pulse_signal, _ = dataset.get_unisens_entry(subject, scenario, VitalSigns.pulse)
        pulse_signal = resample(pulse_signal, self.frames_per_video)

        rr_signal, _ = dataset.get_unisens_entry(subject, scenario, VitalSigns.thorax_abdomen)
        rr_signal = resample(rr_signal, self.frames_per_video)

        rr_signal = preprocessing.standard_processing(rr_signal, self.frames_per_video)
        pulse_signal = preprocessing.standard_processing(pulse_signal, self.frames_per_video)

        return rr_signal, pulse_signal

    def __getitem__(self, index):
        """Get the batch at position index"""

        start = index * self.batch_size
        end = min((index + 1) * self.batch_size, len(self.training_scenarios))
        scenarios = self.training_scenarios[start:end]

        x, y = self.__data_generation(scenarios)
        return x, y

    def __data_generation(self, scenarios: list[tuple[str, str]]):
        """Generates data containing batch_size samples"""

        # The model expects a number of frames that is a multiple of frame_depth

        size = (self.frames_per_video // self.frame_depth) * self.frame_depth - self.frame_depth
        size_all = len(scenarios) * size

        # Create matrix to store the resized frames
        resized_frames_training = np.zeros(
            (size_all, self.dim[0], self.dim[1], 3),
            dtype=np.float32)

        # Create matrix to store the normalized frames
        normalized_frames_training = np.zeros(
            (size_all, self.dim[0], self.dim[1], 3),
            dtype=np.float32)

        pules_labels = np.zeros((size_all, 1), dtype=np.float32)
        rr_labels = np.zeros((size_all, 1), dtype=np.float32)

        for index, (subject, scenario) in enumerate(scenarios):
            frames, meta = dataset.get_video_bgr(subject, scenario)
            resized_frames, normalized_frames = preprocess_video_frames(frames, self.dim)

            rr_signal, pules_signal = self.get_ground_truths(subject, scenario)

            # The model expects to be divisible frame_depth
            rr_signal, pules_signal = rr_signal[:size], pules_signal[:size]
            resized_frames, normalized_frames = resized_frames[:size], normalized_frames[:size]

            start_range = index * size
            end_range = (index + 1) * size

            rr_labels[start_range:end_range] = rr_signal.reshape(-1, 1)
            pules_labels[start_range:end_range] = pules_signal.reshape(-1, 1)

            resized_frames_training[start_range:end_range] = resized_frames
            normalized_frames_training[start_range:end_range] = normalized_frames

        output = (resized_frames_training, normalized_frames_training)
        label = (pules_labels, rr_labels)

        return output, label

In [ ]:
import os
import keras
import tensorflow as tf

np.random.seed(100)  # for reproducibility
keras.backend.clear_session()
print(tf.__version__)
print(tf.config.list_physical_devices('GPU'))

In [ ]:
# number of convolutional filters to use
nb_filters1 = 32
nb_filters2 = 64

dropout_rate1 = 0.25
dropout_rate2 = 0.5
learning_rate = 0.001

# number of dense units
nb_dense = 128

nb_epoch = 24
nb_task = 12

# frame_depth for CAN_3D, TS_CAN, Hybrid_CAN
frame_depth = 10

# CAN, MT_CAN, CAN_3D, MT_CAN_3D, Hybrid_CAN, MT_Hybrid_CAN, TS_CAN, MTTS_CAN
temporal = 'MTTS_CAN'

# train with resp or not
respiration = True

In [ ]:
from respiration.extractor.mtts_can.mtts_can import mtts_can
from scipy.io import savemat


def train(img_rows=36, img_cols=36):
    strategy = tf.distribute.MirroredStrategy()

    with strategy.scope():
        batch_size = 6  # 12, 16, 32

        input_shape = (img_rows, img_cols, 3)
        model = mtts_can(
            frame_depth,
            nb_filters1,
            nb_filters2,
            input_shape,
            dropout_rate1=dropout_rate1,
            dropout_rate2=dropout_rate2,
            nb_dense=nb_dense)

        optimizer = keras.optimizers.Adadelta(learning_rate=learning_rate)
        losses = {
            'output_1': 'mean_squared_error',
            'output_2': 'mean_squared_error',
        }
        loss_weights = {
            'output_1': 1.0,
            'output_2': 1.0,
        }

        model.compile(loss=losses, loss_weights=loss_weights, optimizer=optimizer)

        # Create data generators
        training_generator = DataGenerator(
            training_subjects,
            training_scenarios,
            3600,
            (img_rows, img_cols),
            batch_size=batch_size,
            frame_depth=frame_depth,
        )
        validation_generator = DataGenerator(
            test_subjects,
            test_scenarios,
            3600,
            (img_rows, img_cols),
            batch_size=batch_size,
            frame_depth=frame_depth)

        # Checkpoint Folders
        checkpoint_folder = os.path.join('..', 'models', 'mtts_can')
        cv_split_path = os.path.join(checkpoint_folder, 'checkpoint')
        if not os.path.exists(cv_split_path):
            os.makedirs(cv_split_path)

        save_best_callback = keras.callbacks.ModelCheckpoint(
            filepath=os.path.join(cv_split_path, 'last_model.keras'),
            save_best_only=True,
            verbose=1)

        csv_logger = tf.keras.callbacks.CSVLogger(
            filename=os.path.join(cv_split_path, 'train_loss_log.csv'),
        )

        # Model Training and Saving Results
        history = model.fit(
            x=training_generator,
            validation_data=validation_generator,
            epochs=nb_epoch,
            verbose=1,
            shuffle=False,
            callbacks=[csv_logger, save_best_callback],
            validation_freq=4)

        val_loss_history = history.history['val_loss']
        val_loss = np.array(val_loss_history)
        np.savetxt((cv_split_path + '_val_loss_log.csv'), val_loss, delimiter=",")

        score = model.evaluate_generator(generator=validation_generator, verbose=1)

        print('****************************************')
        print('Average Test Score: ', score[0])
        print('PPG Test Score: ', score[1])
        print('Respiration Test Score: ', score[2])
        print('****************************************')
        print('Start saving predicitions from the last epoch')

        yp_train = model.predict(training_generator, verbose=1)
        savemat(
            os.path.join(checkpoint_folder, 'train_best_' + '_cv.mat'),
            mdict={'yptrain': yp_train})

        yp_test = model.predict(validation_generator, verbose=1)
        savemat(
            os.path.join(checkpoint_folder, 'test_best_' + '_cv.mat'),
            mdict={'yptest': yp_test})

        print('Finish saving the results from the last epoch')

In [ ]:
train()